---
>「人生は選択の連続ですが、一番重要な選択は、あなたの心の内でおこなわれる選択です。」\
>ジョセフ・マーフィー　
---

# AutoEncoder

AutoEncoderは、入力と出力からなるNNであるが、各層のノード数が、途中少なくなるように構成されている

- 入力と出力を同じデータで学習させる、つまり、入力と出力が同じデータとなるように学習が進む

- それは簡単にできるであろうと思うが、途中の層は**ノード数が少ない**ことに注目する

  - この隠れ層が、入力毎とに出力する値(ウエイトではない)が持つ空間(潜在空間:Latent Space)は、学習に使ったデータを**圧縮した**情報を持つといえる

図において、特に圧縮された特徴表現を有する隠れ層が表現する変数空間$z$を潜在変数および潜在空間と呼び、そこに至る圧縮ネットワークをエンコーダ、その後ろの展開ネットワークをデコーダと呼ぶ

<img src="http://class.west.sd.keio.ac.jp/dataai/text/autoencoder.jpg" width=400>

例えば、顔を生成することができるStyleGANは、潜在空間を操作することでいくらでも新しい顔を作ることができる


<img src="http://class.west.sd.keio.ac.jp/dataai/text/image78.gif" width=300>

(話がそれますが)情報匿名化に応用しようというのが研究テーマの一つ
- AutoEncoderにより情報を圧縮し、**情報が圧縮された潜在空間で情報匿名化演算を行う情報匿名化層**を導入し、効率的にかつ、人間の感覚にち近い形で情報を匿名化するアイデアである
- 例えば、顔画像を匿名化する時に、目を隠すのは不自然であるが、潜在空間上で数学的に安全性が示された**近しい顔**に変換することで、自然な匿名化された顔画像が得られるであろう
- 実際にStyleGANで生成した顔画像(この画像は本人の顔から見つけた潜在空間での値の間を直線的に移動させただけ)からわかるように、人間にとって違和感なく**データの中間をとる**ことができる

話としては、分かっていただけたかもしれないが、ここに至るには、まだまだ途中過程がある

まず、AutoEncoderを使ってシンプルに異常検知を実装してみる
- AutoEncoderで学習させる
  - これは全て正解データとする
- その後、テストデータとして実際のデータを入力する
  - このデータには、異常も含まれている

この学習済みモデルを利用すると、異常検知ができる可能性がある

- 結果を用いる方針
  - すると、正解データは**入力と出力の対応の取れた潜在空間で表現できるはずなので**出力が入力と等しくでてくる
  - 一方で、異常データは、そうではないため、出力が入力とは違う形になる
  - さらには、画像などでは**異常個所の誤差が正常箇所よりも大きくなる、また異常の程度で誤差も大きくなる**ように表現できるのではないか？
- 潜在変数を用いる方針
  - 正解データは、**潜在変数が近しいところに集まる**と考えられる
  - 一方で、異常データは外れるため、その差を使って異常を発見する

という発想である

まずは、最初の実装として天気情報を用いて異常気象の日を特定してみよう


## 時系列データの予測と異常検知

気象庁で公開されているオープンデータを利用する
- 過去6年間のデータを取得
- そのうちの5年間のデータを利用
- AutoEncoderで学習し入力と出力が近くなるようにする
- 作成したモデルで異常値を検出

隠れ層に全結合ネットワークを3層設けて、この学習結果を基準とし、大きく外れた値を抽出する

### 気象データの入手

以下の手順で気象データを入手する

- http://www.data.jma.go.jp/gmd/risk/obsdl/ にアクセスする
- 「地点を選ぶ」で、好きな地域を選ぶ。ここでは、神奈川県を選択するが、出身地や行ってみたい都道府県を選択すると良い
- 赤丸のある地域のデータが入手できる
  - 青丸に「日吉」があるが、残念ながらウィンドウ右の観測項目に降水量データしかない
  - 例えば、横浜を選択すると温度計アイコンが得られる
- 次に、「項目を選ぶ」で、日別値、日平均気温を選択する
- 「期間を選ぶ」の連続した期間で表示するを選択し、さらに、例えば、2013年1月1日から、2018年12月31日までを選択する
- 「CSVファイルをダウンロード」をクリックする
- ダウンロードしたdata.csvをいつも通り、左のフォルダアイコンを選択し、フォルダの中に入れる

なお、上記のダウンロードしたデータをそのまま使うには、次のwgetを実行するとよい。


In [ ]:
cuda = "cuda:0"
import os
if not os.path.exists('weatherdata.csv'):
    #!wget "https://drive.google.com/uc?export=download&id=1Om5-a4XKohM9q9fc07GDovci93R0qf-H" -O weatherdata.csv
    !wget https://keio.box.com/shared/static/8ox7nahkgs9go0tber0hfipepgogrzmv  -O weatherdata.csv

このデータは、例のようにweatherdata.csvとして保存されるが、開いてもエラーになる
  - これは、テーブルの形式がラベルが振られてデータが始まるという一般的な形ではないためである

従って、データは整形するので、この時点で、読めなくても問題ない

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

まずは、csvファイルをブラウザでダウンロードし、中身を確認する
  - すると、0行目、1行目、2行目、4行目は不要
  - 3行目はラベルに利用でき、5行目からデータが始まる
  - windows用のデータであるため、漢字コードはshift-jis

これらの情報を基に、pandasの機能を使ってファイルを読み込む
- pandasのcsvロードでは、最初の行が**ラベル**、次の行以降にデータが入っていることを期待しており、これに合わせる
- その中に平均気温というラベルのデータを確認する
   - なお2191個のデータが含まれている

In [ ]:
data = pd.read_csv("weatherdata.csv", skiprows=[0, 1, 2, 4], encoding='shift-jis')
temp_data = data['平均気温(℃)']

このデータを訓練用と、テスト用に分割するが、年境界でさだめるのがよい
- 最後の1年分をテストデータとするため、その境界を探す
  - 単純におおよそ、365*5あたりを見ると良いので、data[1820:1830]とでもしてデータを確認する

In [ ]:
data[1820:1830]

すると、1826行目から新しい年が始まるとわかる
  - ここを境界としてtrain_xとtest_xを分ける
  - **1825までを選ぶ場合は、1足して1826であることに注意**する
  - また、NumPy形式に変換しておく
  - 5年と1年の日数のデータ数があることがわかる

In [ ]:
train_x = np.array(temp_data[:1826])
test_x = np.array(temp_data[1826:])
print(len(train_x), len(test_x))

### データの作成

windows_size = 180としてデータ幅を180個とする
- また、トレーニング用の180日分のデータが複数入るリストを宣言する

In [ ]:
window_size = 180
tmp = []
train_X = []

順番にwindow_size分切り出してtmpに足していく。ここでは、for分を使って順に切り出して、appendでtmpに新しい配列要素として加えている。

最後に、train_Xにtmpをnumpyの配列で保存する。

In [ ]:
for i in range(0, len(train_x) - window_size):
  tmp.append(train_x[i:i+window_size])
train_X = np.array(tmp)
pd.DataFrame(train_X)

RNNの時と同様であるが、100日づつ切り出し、それぞれ1日ずれたデータとした

ここから、ランダムにサンプリングして、トレーニング用のデータセットとする

### モデルの定義

データ幅が180日分であるので、入力は180、そこから128ノードを持つ隠れ層fc1、さらに64ノードを持つ隠れ層fc2、さらに、128ノードを持つ隠れ層fc3、最後に元に戻すために180ノードをもつ出力層fc4を構成する
- つまり、途中64ノードまで次元が削減されている
- 圧縮としては大きすぎだが、あくまでも例である
- xからfc1、fc2までの演算がEncoder、fc2からfc3、fc4までの演算がDecoderである
  - fcは全結合層であり、PyTorchにおけるLinearである

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(180, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 128)
    self.fc4 = nn.Linear(128, 180)
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x
model = Net()
model

### 訓練

損失関数(criterion)と最適化手法(Optimizer)を指定する

また、今回は、学習データから100個ランダムにデータを取得して学習するという処理を1000回繰り返す

- ランダムにデータを取り出す(1)
- 取り出したデータをinput_x配列に加える(2)
- numpyのarrayから、さらにPyTorchのテンソルに変換する (3)
- Optimizerの初期化を行う (4)
  - 最初に勾配を0にする
- input_xから、出力outputを計算する (5)
- 誤差をoutput(出力)と、入力(input_x)の差から求める (6)
- lossを後方伝搬させる (7)
- パラメタを更新する (8)
- 試行1回あたりのロスを累積してエポック全体のロスを求める (9)
- 100回に1回total_lossを表示する (10)



In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(4000):
  total_loss = 0
  input_x = []
  
  for i in range(100):
    index = np.random.randint(0, len(train_X)) #(1)
    input_x.append(train_X[index]) #(2)
  input_x = torch.tensor(np.array(input_x, dtype="float32")) #(3)
  optimizer.zero_grad() #(4)
  output = model(input_x) #(5)
  loss = criterion(output, input_x) #(6)
  loss.backward() #(7)
  optimizer.step() #(8)
  total_loss += loss.item() #(9)
  if (epoch+1) % 200 == 0: #(10)
    print(epoch+1, total_loss)

学習率を調整して、このロスの値が小さくなるようにする。

### 結果表示

トレーニングに使った入力値と出力値をグラフで比較する。

- 入力データを表示する。numpyの配列に変換しておく。
- 出力データを表示する。

In [ ]:
plt.plot(input_x.data[0].numpy(), label='input')
plt.plot(output.data[0].numpy(), label='output')
plt.legend(loc="upper left")

### 異常値検出

テストデータを用いてoutputを求め比較する

In [ ]:
input_x = []
test_X = []
input_x.append(test_x[0:180])
input_x.append(test_x[180:360])
test_X = np.array(input_x, dtype="float32")
input_test = torch.tensor(test_X, requires_grad = False)
model.eval()
with torch.no_grad():
  output = model(input_test)
pd.DataFrame(test_X)

In [ ]:
plt.plot(test_X.flatten(), label="original")
plt.plot(output.data.numpy().flatten(), label="predicted")
plt.legend(loc="upper right")

予測値を求めて、二乗誤差を計算、正規化する

In [ ]:
test = test_X.flatten()
predict = output.data.numpy().flatten()
total_score = []
for i in range(0, 360):
  diff = test[i] - predict[i]
  score = pow(diff, 2)
  total_score.append(score)
total_score = np.array(total_score)
max_score = np.max(total_score)
total_score = total_score / max_score

どの程度の誤差があるかをグラフで表示する
  - 例えば、閾値を0.5とすると、その日は異常値であるとわかる。

In [ ]:
plt.plot(total_score)

ここまでは、なんら従来と変わらないであろう

# MNISTによるAutoEncoder

このあと、CVAEと呼ばれる発展版を学ぶため、比較のため、MNISTでもAEを実装しておく

また、潜在空間について理解を深める

セルの実行でエラーが出た時は、ランタイムを再起動して最初からやり直す必要があるかもしれないので注意すること
- リセットではない
- 例えばパラメタを変更する、記述を修正するなどして、再度試す場合は注意すること

今回はfrom importを多く用いて、記述性を挙げている
- コード記述上は、こちらの方が短く済む
- 但し、一度しか使わない内容について用いるのは考えるべきところ

In [ ]:
import os
import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
# GPU存在のチェック
device = torch.device(cuda if torch.cuda.is_available() else "cpu")
print(device)
#num_epochs = 100
num_epochs = 20
batch_size = 128
learning_rate = 0.001
if not os.path.exists('mydata'):
  os.mkdir('mydata')

データ変換関数で正規化を行う
- MNISTはToTensor()により[0, 1]の値として得られる
  - コメントアウトしているが、外すことで[-1, 1]の範囲に変換する
  - コメントアウトを外す場合、`def to_img(x):`についてもコメントを外して[0, 1]の値に変換しなおす必要がある
    - コメントアウトを忘れると画像が白飛びする
  - 変換しなくても学習は進み、その方がロスは小さくなる傾向にある

In [ ]:
img_transform = transforms.Compose([
  transforms.ToTensor(),
#  transforms.Normalize((0.5), (0.5))
])
train_dataset = MNIST('mydata', download=True, transform=img_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

AutoEncoderモデルを定義する
- 内部空間$z$を利用するため、Encoder と Decoder を独立して定義し、後でDecoderのみ利用できるようにする
- EncoderとDecoderで$w$や$b$などのパラメータは独立させて個別に学習させる

EncoderとDecoderは完全に対象な形としている
- これが必須というわけではないが、AutoEncoderの思想では対象が望ましいであろう

全結合網で784次元から $\rightarrow$ 128 $\rightarrow$ 64 $\rightarrow$ 12 $\rightarrow$ 2次元まで圧縮している
- 2次元では平面上に描画できるため視覚化しやすい
- Decoderは対称に構築している

最終段はいろいろ選んで遊んでみると良いであろう
- `transforms.Normalize((0.5), (0.5))`する場合は入力画像を[-1, 1]に標準化していることから、出力の範囲が [-1, 1]である活性化関数tanhを利用する
  - とはいえ、実はどちらもそれほど変わらない
- 実はLeRU最強であり、実験では最も素早くロスが減少した


In [ ]:
class AE(nn.Module):
  def __init__(self):
    super(AE, self).__init__()
    self.encoder = nn.Sequential(
      nn.Linear(28 * 28, 128),
      nn.ReLU(True),
      nn.Linear(128, 64),
      nn.ReLU(True),
      nn.Linear(64, 12),
      nn.ReLU(True),
      nn.Linear(12, 2))
    self.decoder = nn.Sequential(
      nn.Linear(2, 12),
      nn.ReLU(True),
      nn.Linear(12, 64),
      nn.ReLU(True),
      nn.Linear(64, 128),
      nn.ReLU(True),
      nn.Linear(128, 28 * 28),
#      nn.Tanh() # (*)
#      nn.Sigmoid() # (*)
      nn.ReLU(True) # (*)
    )
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x
model = AE().to(device)
model

変換したテンソルを元の画像[0,1]に戻す関数を定義する
- `#  transforms.Normalize((0.5), (0.5))`と対であり、正規化しない場合は`(*)`の行をコメントアウトする

In [ ]:
def to_img(x):
#  x = 0.5 * (x + 1) # (*)
#  x = x.clamp(0, 1) # (*)
  x = x.view(x.size(0), 1, 28, 28)
  return x

訓練ループで定期的に生成画像を出力している
- 現在エポック数は20であるが、過学習にはならず、じわじわと減少していく
  - 時間に余裕があるならば、思い切って200などにするとよいであろう
- 損失関数は入力と出力の間の平均二乗誤差としている
  - 画像として出力と入力の画素値が一致するように学習がすすむ
最後に、学習したネットワークをモデルとして保存している
- 工夫として最もロスが小さかったモデルのみ覚えるという技も使える
  - ロスを求めて、ロスの最小値を管理し、それよりも小さい場合のみ保存するという手法
  - さらに、GPU用とCPU用を保存している
  - 推奨として、GPU用モデルはGPUで利用し、CPUで利用する場合はCPU用に変更して保存したほうがよい
    - ここでは、両方保存している
    - CPUが選択されるとGPUもCPUモデルが保存される
- 特に保存する必要はないが、こういうこともできるということで

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
    lr=learning_rate, weight_decay=1e-5)
loss_list = []
model.train()
for epoch in range(num_epochs):
  for data in train_loader:
    img = data[0]
    x = img.view(img.size(0), -1).to(device)
    xhat = model(x)
    # 出力画像（再構成画像）と入力画像の間でlossを計算
    loss = criterion(xhat, x)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # logging
    loss_list.append(loss.data.item())
  print('epoch [{}/{}], loss: {:.4f}'.format(
    epoch + 1, num_epochs, loss.data.item()))
    # 10エポックごとに再構成された画像（xhat）を描画する
  if epoch % 5 == 0:
    pic = to_img(xhat.cpu().data)
    save_image(pic, 'mydata/image_{}.png'.format(epoch))

np.save('mydata/loss_list.npy', np.array(loss_list))
torch.save(model.state_dict(), 'mydata/autoencodergpu.pth')
torch.save(model.to('cpu').state_dict(), 'mydata/autoencodercpu.pth')

実験結果としてまずは学習曲線を示す
- 細かいことだが、横軸はiterationである
- エポック毎に表示する場合はepoch、ミニバッチである場合はiterationとする

In [ ]:
loss_list = np.load('{}/loss_list.npy'.format('mydata'))
plt.plot(loss_list)
plt.xlabel('iteration')
plt.ylabel('loss')
plt.grid()

学習途中で保存した画像を確認する
- 学習が進むにつれ、明瞭な画像が得られていることが確認できるであろう
- 入力画像も保存し、対応させるようにすると興味深いであろう
- さらに学習を進めていくと精細さは向上するが、全体的にくっきりとした画像を得るのは困難である
  - 敵対的生成ネットワーク(Generative Adversarial Networks:GAN)はこの点を加速化できるが、学習の安定化が難しい
  - 先に示した顔画像が明瞭なのは、GANを利用しているためである

In [ ]:
from IPython.display import Image
Image('mydata/image_0.png')

In [ ]:
Image('mydata/image_10.png')

In [ ]:
Image('mydata/image_15.png')

潜在空間の可視化

- model2に学習済みモデルを読み直す
  - 特に必要ないが、こういうこともできるということで
  - `(*)`ではGPUモデルを読み出すようになっているが、実行環境に応じて切り替える必要がある
- テストデータ10000画像をEncoderだけを用いて$z$つまり
潜在空間にマッピングする
- $z$内での各画像の分布するか可視化する
- モデル定義において、EncoderとDecoderを別個に定義しているため`model.encoder()`とすることでエンコーダのみ呼び出すことができる

In [ ]:
test_dataset = MNIST('mydata', download=True, train=False, transform=img_transform)
test_loader = DataLoader(test_dataset, batch_size=10000, shuffle=False)
model2 = AE().to(device) # 784次元ベクトルを2次元ベクトルへ圧縮
model2.eval()
model2.load_state_dict(torch.load('mydata/autoencodergpu.pth')) # (*)
data = iter(test_loader).next()
img = data[0]
x = img.view(img.size(0), -1)
x = x.to(device)
with torch.no_grad():
  z = model2.encoder(x)
z = z.data.cpu().numpy()
z.shape

zは (10000, 2) であることがわかる
- つまり784次元の画像が2次元データに圧縮されている

この概念を説明するのが多様性仮説であり、今回スイスロールを展開する関数を学習したと説明できる
- ただし、乱数を固定していないので、毎回異なるイメージが作成できる
  - するとスイスロールの展開方法・関数は乱数個、つまり無限に存在することになり、これも変な話ではないか？ということになる

In [ ]:
import pylab
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.scatter(z[:, 0], z[:, 1], marker='.', c=data[1].numpy(), cmap=pylab.cm.jet)
plt.colorbar()
plt.grid()

各数字のデータがクラスタになって**混ざらずに(実は一部重なってはいるが)**ばらついて分布している
- つまり、2次元の潜在空間上でも各数字を分離できていることを意味する
  
- 潜在空間の分布が±20程度の範囲に拡散している
  - これはAutoEncoderであるためで、VAEであればN(0, I)内に収まる

さらに時間時間がかさむが、もう少しファンシーな表示を行う

In [ ]:
from random import random
colors = ["red", "green", "blue", "orange", "purple", "brown", "fuchsia", "grey", "olive", "lightblue"]
plt.figure(figsize=(10,10))
points = z[:1000]
for p, l in zip(points, data[1]):
  plt.scatter(p[0], p[1], marker="${}$".format(l), c=colors[l])

さて、可視化はできたが、密集する場所が存在し、密集地域での分布がわかりにくい
- これは、多次元を無理やり2次元に落とし込めた際に、空間の曲げ方が**密度が一定になるように次元削減する・曲げる**という制約項もなく、成り行きで次元圧縮しているためである
- 実際には拡大すれば分離されていることがわかる

# (参考)t-SNE

さて、本授業の範囲外であるが、単純に次元削減したい場合は、t-SNEが利用できる
  - 興味がある場合は調べると良いであろう
  - 2次元/3次元程度に圧縮する場合は有効であるが、複雑な場合は別の方法を選択したほうがよい
  - MNISTはt-SNE向きである
- はっきり言えば、2次元3次元の可視化であればt-SNEを利用するべき

サンプルコードを示すが、よっぽどきれいに分類できていることがわかる
- 似ている文字のグループが近くに存在することもわかる
- ただし逆変換を提供しないため、モーフィングなどの処理はできない

In [ ]:
from sklearn import random_projection
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

digits = datasets.load_digits()

X_TSNEprojected = TSNE(n_components=2, random_state=0).fit_transform(digits.data)
X_Gprojected    = random_projection.GaussianRandomProjection(n_components=2).fit_transform(digits.data)
X_Sprojected    = random_projection.SparseRandomProjection(n_components=2).fit_transform(digits.data)

plt.scatter(X_TSNEprojected[:,0], X_TSNEprojected[:,1], c=digits.target,alpha=0.5, cmap='rainbow')
plt.colorbar()

外れるついでに、どんどん道にそれる

Google Colaboratoryの真のパワーを見てみよう

まず、tensorboardXを導入する
- 著名な可視化ツール

なお、現状でPyTorch1.1よりtensorboardに正式対応しており、`pip install tensorboard`でほぼ同様の操作が可能となっている

今回は、PyTorchとの連携は行わず、単体利用であるが、本来は連携して利用することでさらに威力を発揮する
- 次のようにコードを記述する
```
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)
model.fit(x_train, y_train, epochs=5, callbacks=[tf_callback])
model.evaluate(x_test,  y_test, verbose=2)
```
コールバック関数としてTensorboardを登録し、model.fit、model.evaluateを呼び出すことで、リアルタイムに学習を進めながらグラフ化することができるので便利
  - 複数の情報を纏めて表示することもできる
- 大規模学習の際には便利

In [ ]:
!pip install tensorboardX

ログファイルの保存場所を指定する
- pythonの中から指定するために、SummmaryWriteを導入して指定する

In [ ]:
from tensorboardX import SummaryWriter
writer1 = SummaryWriter()
writer2 = SummaryWriter(logdir='logs/image')
writer3 = SummaryWriter(comment='loss function')

PyTorchを導入するが、あくまでもMNISTのデータをとるためと、データを食わせるため

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='mydata',
                               train=True,
                               transform=transform,
                               download=True) 
test_dataset = datasets.MNIST(root='mydata',
                              train=False,
                              transform=transform)
batch_size = 1000
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
mages, labels = next(iter(train_loader))

データを保存する

In [ ]:
images, labels = next(iter(train_loader))
mat_img = images.view(-1, 28*28)
meta_labels = [str(x) for x in labels.numpy().tolist()]
with SummaryWriter(logdir='logs/projector') as w:
  w.add_embedding(mat_img, metadata=meta_labels, label_img=images)

さぁ、お楽しみだ
- PROJECTORが選択できる(出ない場合はリロードすること)
- T-SNEを選んで楽しもう
- テレビや動画でこのような画面をよく見るであろう
- 何も偉くないのだ
  - さも偉そうに見せて、お金を取ってくる、これがコンサルだ、合法詐欺師だ

In [ ]:
%load_ext tensorboard
#TensorBoard起動（表示したいログディレクトリを指定）
%tensorboard --logdir=logs

# Variational Autoencoder(VAE)

AutoEncoderが目指すところは理解できたであろう

しかしながら、先ほどの顔のモーフィングのように**連続して移動できる**保証はどこにあるのだろうか？
  - 潜在空間を埋め尽くすように、各顔の特徴がマッピングされるとは限らないのではないか？
  - その通りで、実際にAutoEncoderは学習させてうまく生成できるが、圧縮できたという以上に使い道が実はそれほどない
  - 潜在空間をみても、隙間が多く、その隙間に位置する潜在空間ではどのような文字が生成されるかがわからない
    - そもそも文字ではなくなってしまうが

<img src="http://class.west.sd.keio.ac.jp/dataai/text/aefig.png" width=300>

この図のように、潜在空間をどのようにするか、については制御していない

そこで、VAEは潜在変数$z$に確率分布として例えば$z∼N(0,1)$を想定し($N$は多変量ガウス分布)、$z$に無理やりノイズを入れる手法であり、結果として特徴量の連続性を維持したり、潜在変数がスパースになるのを防ぐことを狙う

なぜそれでよいのかについて、以下説明する

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaefig.png" width=300>

VAEでは、Encoderにおいて平均ベクトル$\mu$と分散ベクトル$\sigma$を学習し、これらの平均ベクトル、分散ベクトルから得られる多変量ガウス分布から潜在変数$z$をサンプリングする
- つまり、$z∼N(\mu,\sigma)$とする
- このベクトル$z$が次元圧縮後のベクトルとなる

このようなアイデアは常に「学習できること」が重要であり、単純にこのままのモデルでは誤差逆伝播ができない
- $\mu$と$\sigma$で得られるガウス分布から$z$をサンプリングする点で、逆伝播の演算ができなくなっている


## Reparametrization Trick

逆伝播の演算ができない問題を解決するため、実際のVAEではReparametrization Trickが用いられている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaetrick.png" width=300>

$z$をサンプリングで求めずに、
$z=\mu+\epsilon\sigma(\epsilon∼N(0,I))$と近似する

実際のネットワーク構造は下記のようになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/reptrik.webp" width=300>

$z∼N(\mu,\sigma)$を直接扱わず、$\epsilon∼N(0,I)$にてノイズを発生させ、図に示すように$z=\mu(X)+\epsilon \cdot \sigma(X)$という形でつなげることで、**確率変数を避けて青い矢印を逆にたどって誤差逆伝播法を適用する**という工夫がなされている
  - 黒いパスは逆伝播では利用しない

## Regularization Parameter

VAEはAutoEncoderと同様に元の画像を復元するように学習させる
- AutoEncoderではReconstraction Error(復元誤差)を単純に用いる

VAEではReconstraction Errorだけでなく、Regularization Parameter(正則化項)の学習も行う

Regularization Parameterの式は$D_{KL}$をKLダイバージェンスとして次の通り
$$
RegLoss=−D_{KL}(N(\mu,\sigma)|N(0,I))
$$
- つまり、平均ベクトル$\mu$と分散ベクトル$\sigma$がなるべく原点を中心としたベクトルになるように学習させるための項が追加される
- この項により、VAEによって次元削減されたベクトルは、原点を中心に連続的に変化するベクトルとなる

# Conditional Variational Auto Encoder (CVAE)

CVAEは、VAEに対して正解ラベルも付与して学習を行う手法
- Encoder、Decoder両方に正解ラベルを付与して学習させている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/cvaefig.png" width=300>

その利点は次の通り
- Encoderで次元削減するとき、画像データだけでなくそのラベルを反映させることができる
- Decoderでデータ生成するとき、欲しいデータの状態を指定することができる
- 全データに正解がある必要はなく、半教師ありで次元削減ができる

## 具体的な効果

例えば、MNISTを$z$が2次元のVAEで学習させることを考える

- 訓練データのデータセットを入れて潜在変数を求めこれを順番に座標系にプロットする
- VAEの場合、訓練データのMNISTデータセットが2次元の正規分布に従う円の上に散らばるようになる
  - そうなるように確率分布を想定した
- すると**同じクラスラベルのデータが近いところに集まっている**ことが確認でき、点で見えているが、点と点の間もそれにふさわしい画像を表す$z$で埋め尽くされているであろうと考えられる
  - VAEは正規分布に従う乱数を学習時に取り入れることで、この乱数の乱雑性により似た形状を近くに寄せる効果を与える
  - 同じ画像を同じ値で表現せず、敢えて毎回散らして学習させることで、**この画像の表現範囲はこの辺り**と明確に定まらないようにして、あえてぼんやり学習させる
  - すると、それぞれのぼんやりの重なりが現れるようになり、連続的に推移できるようになる

MNISTをVAEで学習させた例が次の図で、図の点は訓練データを入力した際の$z$の値のプロットを表している

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaemnist.png" width=300>

すると、先ほどの顔匿名化と同じことが起こる(というか、顔匿名化はこの技術の応用なのだが)
- 例えば、0から7に至る線分上の点、つまり内分点を順番に求めて、デコーダだけ使って画像を得ると、次のようになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaemnistmove.png" width=300>

実際に得られる出力は次の通り

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaemove.gif" width=100>

0から滑らかに、6、2、8、9、4、7のように推移するのがわかる
- 4と9の間を通るので、4と9が混ざったような数字が出るのも、その通り

さらに$z$の空間に直接格子状に点を作って次々と代入すると、マッピング図が得られ、この上の0から7へ移動させたことが明確となる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/vaemnistmap.webp" width=500>

マップ上で、0から1、1から2、2から3と動かしていくと、数字が滑らかに変化するような動画が得られる
- ストップウォッチや時計に利用すると、若干カッコいい？
- 芸術家でさえAIを使う時代なのだ


## 多様体仮説

そもそも、なぜこういうことができるのか？を根本的に説明することは実はできていないといってよい
- なぜか現実世界はそういうスパースな状況に見えて、集まっているのである
- 宇宙の大構造のようなイメージか？

その説明の一つが次の**多様体仮説**である

- 多様な次元をもつデータがあるとしても、そのデータは少ない次元で表現できたとする(学習できた)

  - このとき、意味のあるデータ、つまり訓練データにおける本質をとらえたデータは、高次元のデータの中でも局所的に固まっている
  - つまり、高次元で情報を見る(例えば、画像でMAEを算出する)よりも、その少ない次元での距離を測る方が類似しているものが見つかるといえる
- 例えば、像とハスキー犬と白柴犬の写真があったとする
  - 画素でMAE計算すれば、色の近い像とハスキー犬が近いと判断してしまうであろう
  - 潜在空間$z$でみれば、ハスキー犬と白柴犬の方が近い
    - 同様に動物の特徴、例えば、足の長さや耳の長さ、胴体の形などの近しいものが並んでいくであろう
    - そうであるならば、$z$上の距離が、本質をついた差を表現していないか？ということになる

次の図は、多様体仮説と同じスイスロール理論について説明した図である
- 元の空間ではスパースで、その上での距離に意味はない
- ところがスイスロールのクリームの上では連続でその上で距離を測ることに意味がある、という考え方
- 実はカーネルトリックなども、このスイスロールの考え方に基づいており、**うまい関数を用いて**このロールを開く変換が構築できれば、距離は簡単に求まる、つまり、線形処理できる、という考え方になる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/swissroll.webp" width=500>


# CVAEの実装

先にTensorBoardを見てしまうと、ビジュアル的にCVAEはしょぼく感じるかもしれないが、気合を入れなおして張り切って学ぼう

今回の目標は、「人の癖をくみ取ったフォントの自動生成」である

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
# ハイパーパラメータ集
SEED = 0
CLASS_SIZE = 10
BATCH_SIZE = 256
ZDIM = 16
NUM_EPOCHS = 50
# GPU存在のチェック
DEVICE = torch.device(cuda if torch.cuda.is_available() else "cpu")
print(DEVICE)
# シードの設定
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)   
torch.cuda.manual_seed(SEED)
# データセットとデータローダの準備
train_dataset = torchvision.datasets.MNIST(
  root='mydata', train=True, transform=transforms.ToTensor(), download=True,
)
train_loader = torch.utils.data.DataLoader(
  dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=0
)

## モデルの定義と処理記述

- AutoEncoderでは無理やり2次元に押し込めた
  - さすがにそれは無理があるので、今度は16次元への圧縮
  - ただし、そこから
- $z$の部分に、mean、lnvarを計算している部分がある
  - これが、VAEとCVAEに共通する部分である
```
self._to_mean = nn.Linear(hidden_units, zdim)
self._to_lnvar = nn.Linear(hidden_units, zdim)
```
および
```
mean = self._to_mean(h)
lnvar = self._to_lnvar(h)
```
において、先の説明通りの結合が構成されている
  - 残りの結合は、外部の記述上に存在する

入出力でCVAEならではの記述がされている
- `def encode(self, x, labels):`は、imageとlabelを引数とし、$\sigma$、$\mu$を返す
- `def decode(self, z, labels):`は、`z = mean + std * epsilon`とラベルを入力とし、imageとlabelを返り値としている
- これらがCVAEの部分である

In [ ]:
class CVAE(nn.Module):
  def __init__(self, zdim):
    super().__init__()
    self._zdim = zdim
    self._in_units = 28 * 28
    hidden_units = 512
    self._encoder = nn.Sequential(
      nn.Linear(self._in_units + CLASS_SIZE, hidden_units),
      nn.ReLU(inplace=True),
      nn.Linear(hidden_units, hidden_units),
      nn.ReLU(inplace=True),
    )
    self._to_mean = nn.Linear(hidden_units, zdim)
    self._to_lnvar = nn.Linear(hidden_units, zdim)
    self._decoder = nn.Sequential(
      nn.Linear(zdim + CLASS_SIZE, hidden_units),
      nn.ReLU(inplace=True),
      nn.Linear(hidden_units, hidden_units),
      nn.ReLU(inplace=True),
      nn.Linear(hidden_units, self._in_units),
      nn.Sigmoid()
    )
  def encode(self, x, labels):
    in_ = torch.empty((x.shape[0], self._in_units + CLASS_SIZE), device=DEVICE)
    in_[:, :self._in_units] = x
    in_[:, self._in_units:] = labels
    h = self._encoder(in_)
    mean = self._to_mean(h)
    lnvar = self._to_lnvar(h)
    return mean, lnvar
  def decode(self, z, labels):
    in_ = torch.empty((z.shape[0], self._zdim + CLASS_SIZE), device=DEVICE)
    in_[:, :self._zdim] = z
    in_[:, self._zdim:] = labels
    return self._decoder(in_)

PyTorchには、on-hotをワンライナーで構成する関数が準備されているので、今回はそれを利用する

モデルの定義を行い、最適化にAdamを指定する

In [ ]:
def to_onehot(label):
  return torch.eye(CLASS_SIZE, device=DEVICE, dtype=torch.float32)[label]
model = CVAE(ZDIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

実際の処理は次の通り

GPUに持っていける(効率の良い)関数とそうでない関数がある
- 乱数はもっていけないので、CPU側で計算する
- また、ロス計算で`mean`と`lnvar`を利用する

ここでは、**バイナリクロスエントロピー(2値交差エントロピー)**が利用されている
- MSEはそのまま誤差が求まるが、ここでは、ミニバッチそれぞれの中で異なる乱数値が用いられていることから、それらを踏まえてバラバラに計算する必要がある
- そこで、バイナリクロスエントロピーを用いることで、テンソルで個別に計算、つまりベクトルの形で処理を進め、最後にその平均を求めている
- 各画素を多クラスと見做している

VAEでのロスの計算方法にも注目する
>```
loss = (-1 * kld + bce).mean()
```

と記述されている
- kldはRegularization Parameterに相当し、平均ベクトルμと分散ベクトルσがなるべく原点を中心としたベクトルになるように学習させるための項である
- bceはReconstraction Errorに相当し、画像としての再現度合いを評価する

さて、`mean, lnvar = model.encode(x, labels)`によりZDIM(ここでは16次元)の`mean`と`lnvar`が得られる
- これを、`std = lnver.exp().sqrt()`として、やはり16次元の$\sqrt{e^{x_i}}$を計算
- `epciron`にやはり16次元の平均0分散1の正規分布の乱数を入れる
- `z = mean + std * epsilon`として潜在空間$z$を求める

CVAEの図のパスそのものが配列で構成されているイメージ

In [ ]:
model.train()
for e in range(NUM_EPOCHS):
  train_loss = 0
  for i, (images, labels) in enumerate(train_loader):
    labels = to_onehot(labels)
    x = images.view(-1, 28*28*1).to(DEVICE) # 画像を1次元へ
    mean, lnvar = model.encode(x, labels) # ラベルと一緒にエンコーダへ
    std = lnvar.exp().sqrt()
    epsilon = torch.randn(ZDIM, device=DEVICE) # 乱数を生成
    z = mean + std * epsilon # 潜在変数を変換
    y = model.decode(z, labels) # デコード
    # ロスを計算
    kld = 0.5 * (1 + lnvar - mean.pow(2) - lnvar.exp()).sum(axis=1)
    bce = F.binary_cross_entropy(y, x, reduction='none').sum(axis=1)
    loss = (-1 * kld + bce).mean()
    # Update model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item() * x.shape[0]
  print(f'epoch: {e + 1} epoch_loss: {train_loss/len(train_dataset)}')

## CVAEを用いた実験

CVAEの特徴として、正解ラベルも一緒に学習されているため、
- zとして適当なランダムの値をいれる
  - VAEの特徴により密に原点付近に集まるように分布している
- ワンホットにした文字としての数字について、$z$を乱数で与えているため、様々な数字を自動生成する
  - GANのGeneratorと同じ感覚
  - もっともらしい、(誰も書いていない)嘘の数字をいくらでも作れるようになる

次の手順で試してみよう
- `getlabel=0`を修正
- セルを実行することで25個の数字を作成し表示させる
- 様々な数字の上にキャプションがある
  - Gen(作成した数字)[ID]であり、このIDは次で利用する

In [ ]:
getlabel = 0
NUM_GENERATION = 25
model.eval()
allz = []
fig, ax = plt.subplots(5, 5, figsize=(5,6.5))
for i in range(NUM_GENERATION):
  z = torch.randn(ZDIM, device=DEVICE).unsqueeze(dim=0)
  label = torch.tensor([getlabel], device=DEVICE)
  with torch.no_grad():
    y = model.decode(z, to_onehot(label))
  y = y.reshape(28, 28).cpu().detach().numpy()
  # Save image
  ly = int(i/5)
  lx = i%5
  ax[ly, lx].imshow(y, cmap='gray')
  ax[ly, lx].set_title(f'G({label.cpu().detach().numpy()[0]})[{i}]')
  ax[ly, lx].tick_params(
    labelbottom=False,
    labelleft=False,
    bottom=False,
    left=False,
  )
  allz.append(z)
plt.show()

気になる形状の数字(フォントといってよいだろう)があれば、そのIDを調べる
- 細い、太い、斜め、三角、薄い、など特徴的なフォントを選ぶとわかりやすい

このIDがどういう潜在空間を持っているかを調べる
- つまり、これがその書体の特徴量であり、ID、固有番号である
- ここでは、先に示した通り16次元のベクトルで表現されている

In [ ]:
z = allz[15]
z

次に、`genseries`関数を定義している
- この関数は、潜在空間の値を用いて、その書体で他の数字を生成することができる
- 次のセルを実行することで、選択したIDの形を維持した他の数字が表れるであろう

In [ ]:
def genseries(z):
  model.eval()
  fig, ax = plt.subplots(2, 5, figsize=(5,2.5))
  for label in range(CLASS_SIZE):
    with torch.no_grad():
      y = model.decode(z, to_onehot(label))
    y = y.reshape(28, 28).cpu().detach().numpy()
    # Save image
    ly = int(label/5)
    lx = label%5
    ax[ly, lx].imshow(y, cmap='gray')
    ax[ly, lx].set_title(f'Gen({label})')
    ax[ly, lx].tick_params(
      labelbottom=False,
      labelleft=False,
      bottom=False,
      left=False,
    )
  plt.show()
genseries(z)

## 手書き文字の潜在空間とそれを用いた数字生成

さて、次に自分の手書き文字の潜在空間を調べる

次の手順で試してみよう
- まず「それなりにちゃんとした書体で手書き数字を準備する」
  - 紙に書いて写メでもよいし、ペイントブラシなどのアプリで描いてもよい
  - そのデータをgifやpngで保存する(ファイル名は英語のほうが問題が少ない)
  - そのファイルを左のフォルダの中にドラッグする
  - ドラッグしたら、次のセルの中の
 ```
 my_fig = Image.open("mytest.jpg")
 ```
 の部分にあるファイル名を、保存したファイル名で書き換える、**さらに**
 ```
 my_label = 1
 ```
 を**書いた数字の番号に書き換える**
  - セルを実行すると画面に表示される

  - さらにその次のセルを実行すると、保存したファイルの潜在空間$z$が表示される

In [ ]:
if not os.path.exists('mytest.jpg'):
  #!wget "https://drive.google.com/uc?export=download&id=1o1xxp0AtkCyjTTLW_oyqSKVHX5lLAJuN" -O mytest.jpg
  !wget https://keio.box.com/shared/static/n2sdt9rvju9b6tcsuer9wogy8770c5is -O mytest.jpg

In [ ]:
from PIL import Image
from PIL import ImageOps
my_label = 1
from scipy.ndimage import center_of_mass
my_fig = Image.open("mytest.jpg")
my_img = ImageOps.invert(my_fig.resize((28, 28)).convert("L"))
plt.figure(figsize=(0.8, 0.8))
plt.tick_params(
  labelbottom=False,
  labelleft=False,
  bottom=False,
 left=False,
)
plt.imshow(my_img, cmap='gray') 

次のセルを実行すると、保存したファイルの潜在空間zが表示される

In [ ]:
my_input = torch.tensor(np.array(my_img)/256)
x = my_input.view(1, 28*28).to(DEVICE)
label = torch.tensor(np.array(my_label)).clone().to(DEVICE)
with torch.no_grad():
  mean, _ = model.encode(x, to_onehot(label))
z = mean
print(f'z = {z}')

保存した画像ファイルの潜在空間zが表示されるということは、それをもとに他の数字も作成できるということである
- 先ほど作成した関数genseriesを利用して表示する

In [ ]:
genseries(z)

# 課題B (AE)

CVAEの「手書き文字の潜在空間とそれを用いた数字生成」に従って、自分の手書きクセ字で数字を記載、写メるなどして画像を保存し、自分のクセ字をもとにした数字文字を生成しなさい

レポートには次の内容をそろえること
- 実行可能なノートブック
- 自分のクセ字
- 生成した数字の一覧(0から9まで)

<font color='red'>皆さん個人のクセ字を用いるので、同じ潜在空間表現にはなりえません。</font>